In [ ]:
from lec_utils import *

In [ ]:
def show_paradox_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vQuT3iKLsCvkBhicHF4hhQi2n1pmE4pfk1bf2_T8pmjOOCpf9G5DKPlsKVR5jy8Bgsd_9J4dUY1OXhm/pubembed?start=false&loop=false"
    width = 960
    height = 569
    display(IFrame(src, width, height))

In [ ]:
# Pandas Tutor setup
%reload_ext pandas_tutor
%set_pandas_tutor_options {"maxDisplayCols": 8, "nohover": True, "projectorMode": True}

# Lecture 3 – Aggregating, Simpsons Paradox


In [ ]:
import seaborn as sns

penguins = sns.load_dataset("penguins").dropna()
penguins

## Other `DataFrameGroupBy` methods


### Split-apply-combine, revisited

When we introduced the split-apply-combine pattern, the "apply" step involved **aggregation** – our final DataFrame had one row for each group.

<center><img src="imgs/image_0.png" width=40%></center>

Instead of aggregating during the apply step, we could instead perform a:

- **Transformation**, in which we perform operations to every value within each group.

- **Filtration**, in which we keep only the groups that satisfy some condition.


### Transformations

Suppose we want to convert the `'body_mass_g'` column to to z-scores (i.e. standard units):

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$


In [ ]:
def z_score(x):
    return (x - x.mean()) / x.std(ddof=0)

In [ ]:
z_score(penguins["body_mass_g"])

### Transformations within groups

- Now, what if we wanted the z-score within each group?

- To do so, we can use the `transform` method on a `DataFrameGroupBy` object. The `transform` method takes in a function, which itself takes in a Series and returns a new Series.

- A transformation produces a DataFrame or Series of the same size – it is **not** an aggregation!


In [ ]:
z_mass = penguins.groupby("species")["body_mass_g"].transform(z_score)
z_mass

In [ ]:
penguins.assign(z_mass=z_mass)

In [ ]:
display_df(penguins.assign(z_mass=z_mass), rows=8)

Note that above, penguin 340 has a larger `'body_mass_g'` than penguin 0, but a lower `'z_mass'`.

- Penguin 0 has an above average `'body_mass_g'` among `'Adelie'` penguins.
- Penguin 340 has a below average `'body_mass_g'` among `'Gentoo'` penguins. Remember from earlier that the average `'body_mass_g'` of `'Gentoo'` penguins is much higher than for other species.


In [ ]:
penguins.groupby("species")["body_mass_g"].mean()

### Filtering groups

- To keep only the groups that satisfy a particular condition, use the `filter` method on a `DataFrameGroupBy` object.

- The `filter` method takes in a function, which itself takes in a DataFrame/Series and return a single Boolean. The result is a new DataFrame/Series with only the groups for which the filter function returned `True`.

For example, suppose we want only the `'species'` whose average `'bill_length_mm'` is above 39.


In [ ]:
(
    penguins.groupby("species").filter(
        lambda df: df["bill_length_mm"].mean() > 39
    )
)

No more `'Adelie'`s!


Or, as another example, suppose we only want `'species'` with at least 100 penguins:


In [ ]:
(penguins.groupby("species").filter(lambda df: df.shape[0] > 100))

No more `'Chinstrap'`s!


<div class="alert alert-warning">
    <h3>Question 🤔 </h3>

Answer the following questions about grouping:

- In `.agg(fn)`, what is the input to `fn`? What is the output of `fn`?
- In `.transform(fn)`, what is the input to `fn`? What is the output of `fn`?
- In `.filter(fn)`, what is the input to `fn`? What is the output of `fn`?
</div>


### Grouping with multiple columns

When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.


In [ ]:
penguins

In [ ]:
species_and_island = penguins.groupby(["species", "island"])[
    ["bill_length_mm", "body_mass_g"]
].mean()
species_and_island

### Grouping and indexes

- The `groupby` method creates an index based on the specified columns.
- When grouping by multiple columns, the resulting DataFrame has a `MultiIndex`.
- Advice: When working with a `MultiIndex`, use `reset_index` or set `as_index=False` in `groupby`.


In [ ]:
species_and_island

In [ ]:
species_and_island["body_mass_g"]

In [ ]:
species_and_island.loc["Adelie"]

In [ ]:
species_and_island.loc[("Adelie", "Torgersen")]

In [ ]:
species_and_island.reset_index()

In [ ]:
(
    penguins.groupby(["species", "island"], as_index=False)[
        ["bill_length_mm", "body_mass_g"]
    ].mean()
)

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>

Find the most popular `Male` and `Female` baby `Name` for each `Year` in `baby`. **Exclude** `Year`s where there were fewer than 1 million births recorded.

</div>


In [ ]:
baby_path = Path("data") / "baby.csv"
baby = pd.read_csv(baby_path)
baby

In [ ]:
# Your code goes here.

## Pivot tables using the `pivot_table` method


### Pivot tables: an extension of grouping

Pivot tables are a compact way to display tables for humans to read:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Sex</th>
      <th>F</th>
      <th>M</th>
    </tr>
    <tr>
      <th>Year</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2018</th>
      <td>1698373</td>
      <td>1813377</td>
    </tr>
    <tr>
      <th>2019</th>
      <td>1675139</td>
      <td>1790682</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>1612393</td>
      <td>1721588</td>
    </tr>
    <tr>
      <th>2021</th>
      <td>1635800</td>
      <td>1743913</td>
    </tr>
    <tr>
      <th>2022</th>
      <td>1628730</td>
      <td>1733166</td>
    </tr>
  </tbody>
</table>


- Notice that each value in the table is a sum over the counts, split by year and sex.
- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**


### `pivot_table`

The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:

```py
df.pivot_table(index=index_col,
               columns=columns_col,
               values=values_col,
               aggfunc=func)
```

The resulting DataFrame will have:

- One row for every unique value in `index_col`.
- One column for every unique value in `columns_col`.
- Values determined by applying `func` on values in `values_col`.


In [ ]:
last_5_years = baby.query("Year >= 2018")
last_5_years

In [ ]:
last_5_years.pivot_table(
    index="Year",
    columns="Sex",
    values="Count",
    aggfunc="sum",
)

In [ ]:
# Look at the similarity to the snippet above!
(last_5_years.groupby(["Year", "Sex"])[["Count"]].sum())

### Example

Find the number of penguins per `'island'` and `'species'`.


In [ ]:
penguins

In [ ]:
penguins.pivot_table(
    index="species",
    columns="island",
    values="bill_length_mm",  # Choice of column here doesn't actually matter!
    aggfunc="count",
)

Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.

We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.


In [ ]:
penguins.pivot_table(
    index="species",
    columns="island",
    values="bill_length_mm",
    aggfunc="count",
    fill_value=0,
)

### Granularity, revisited

Take another look at the pivot table from the previous slide. Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

What is the granularity of the DataFrame below?


In [ ]:
penguins.pivot_table(
    index="species",
    columns="island",
    values="bill_length_mm",
    aggfunc="count",
    fill_value=0,
)

### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".
- Other DataFrame reshaping methods:
  - `melt`: Un-pivots a DataFrame. Very useful in data cleaning.
  - `pivot`: Like `pivot_table`, but doesn't do aggregation.
  - `stack`: Pivots multi-level columns to multi-indices.
  - `unstack`: Pivots multi-indices to columns.
  - Google and the documentation are your friends!


## Distributions


### Joint distribution

When using `aggfunc='count'`, a pivot table describes the **joint distribution** of two categorical variables. This is also called a **contingency table**.


In [ ]:
counts = penguins.pivot_table(
    index="species",
    columns="sex",
    values="body_mass_g",
    aggfunc="count",
    fill_value=0,
)
counts

We can normalize the DataFrame by dividing by the total number of penguins. The resulting numbers can be interpreted as **probabilities** that a randomly selected penguin from the dataset belongs to a given combination of species and sex.


In [ ]:
joint = counts / counts.sum().sum()
joint

### Marginal probabilities

If we sum over one of the axes, we can compute **marginal probabilities**, i.e. unconditional probabilities.


In [ ]:
joint

In [ ]:
# Recall, joint.sum(axis=0) sums across the rows,
# which computes the sum of the **columns**.
joint.sum(axis=0)

In [ ]:
joint.sum(axis=1)

For instance, the second Series tells us that a randomly selected penguin has a 0.36 chance of being of species `'Gentoo'`.


### Conditional probabilities

Using `counts`, how might we compute conditional probabilities like $$P(\text{species } = \text{"Adelie"} \mid \text{sex } = \text{"Female"})?$$


In [ ]:
counts

$$
\begin{align*}
P(\text{species} = c \mid \text{sex} = x) &= \frac{\# \: (\text{species} = c \text{ and } \text{sex} = x)}{\# \: (\text{sex} = x)}
\end{align*}
$$

<details>
    <summary>➡️ Click <b>here</b> to see more of a derivation.</summary>
$$\begin{align*}
P(\text{species} = c \mid \text{sex} = x) &= \frac{P(\text{species} = c \text{ and } \text{sex} = x)}{P(\text{sex = }x)} \\
&= \frac{\frac{\# \: (\text{species } = \: c \text{ and } \text{sex } = \: x)}{N}}{\frac{\# \: (\text{sex } = \: x)}{N}} \\
&= \frac{\# \: (\text{species} = c \text{ and } \text{sex} = x)}{\# \: (\text{sex} = x)}
\end{align*}$$
</details>


**Answer**: To find conditional probabilities of **`'species'` given `'sex'`**, divide by **column sums**. To find conditional probabilities of **`'sex'` given `'species'`**, divide by **row sums**.


### Conditional probabilities

To find conditional probabilities of **`'species'` given `'sex'`**, divide by **column sums**. To find conditional probabilities of **`'sex'` given `'species'`**, divide by **row sums**.


In [ ]:
counts

In [ ]:
counts.sum(axis=0)

The conditional distribution of **`'species'` given `'sex'`** is below. Note that in this new DataFrame, the `'Female'` and `'Male'` columns each sum to 1.


In [ ]:
counts / counts.sum(axis=0)

For instance, the above DataFrame tells us that the probability that a randomly selected penguin is of `'species'` `'Adelie'` **given** that they are of `'sex'` `'Female'` is 0.442424.


<div class="alert alert-warning">
    <h3>Question 🤔 </h3>

Find the conditional distribution of `'sex'` given `'species'`.

**_Hint_**: Use `.T`.

</div>


In [ ]:
# Your code goes here.

## Simpson's paradox

<br>

<center><img src="imgs/simpsons.png" width=50%></center>


### Example: Grades

- Two students, Lisa and Bart, just finished their first year at UCSD. They both took a different number of classes in Fall, Winter, and Spring.

- Each quarter, Lisa had a higher GPA than Bart.

- But Bart has a higher overall GPA.

- How is this possible? 🤔


Run this cell to create DataFrames that contain each students' grades.


In [ ]:
lisa = pd.DataFrame(
    [[20, 46], [18, 54], [5, 20]],
    columns=["Units", "Grade Points Earned"],
    index=["Fall", "Winter", "Spring"],
)
lisa.columns.name = "Lisa"  # This allows us to see the name "Lisa" in the top left of the DataFrame.

bart = pd.DataFrame(
    [[5, 10], [5, 13.5], [22, 81.4]],
    columns=["Units", "Grade Points Earned"],
    index=["Fall", "Winter", "Spring"],
)
bart.columns.name = "Bart"

### Quarter-specific vs. overall GPAs

**Note**: The number of "grade points" earned for a course is

$$\text{number of units} \cdot \text{grade (out of 4)}$$

For instance, an A- in a 4 unit course earns $3.7 \cdot 4 = 14.8$ grade points.


In [ ]:
dfs_side_by_side(lisa, bart)

Lisa had a higher GPA in all three quarters.


In [ ]:
quarterly_gpas = pd.DataFrame(
    {
        "Lisa's Quarter GPA": lisa["Grade Points Earned"] / lisa["Units"],
        "Bart's Quarter GPA": bart["Grade Points Earned"] / bart["Units"],
    }
)

quarterly_gpas

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>

</div>

Use the DataFrame `lisa` to compute Lisa's overall GPA, and use the DataFrame `bart` to compute Bart's overall GPA.


In [ ]:
# Helper function to show lisa and bart side-by-side to save screen space
dfs_side_by_side(lisa, bart)

In [ ]:
# Your code goes here.

### What happened?


In [ ]:
(
    quarterly_gpas.assign(
        Lisa_Units=lisa["Units"], Bart_Units=bart["Units"]
    ).iloc[:, [0, 2, 1, 3]]
)

- When Lisa and Bart both performed poorly, Lisa took more units than Bart. **This brought down 📉 Lisa's overall average.**

- When Lisa and Bart both performed well, Bart took more units than Lisa. **This brought up 📈 Bart's overall average.**


### Simpson's paradox

- Simpson's paradox occurs when **grouped data and ungrouped data show opposing trends**.

  - It is named after Edward H. Simpson, not Lisa or Bart Simpson.

- It often happens because there is a hidden factor (i.e. a **confounder**) within the data that influences results.

- **Question**: What is the "correct" way to summarize your data? What if you had to act on these results?


### Example: How Berkeley was _almost_ sued for gender discrimination (1973)

What do you notice?

<center><img src='imgs/berkeley.png' width=70%></center>


In [ ]:
show_paradox_slides()

### What happened?

- The overall acceptance rate for women (30%) was lower than it was for men (45%).

- However, most departments (A, B, D, F) had a higher acceptance rate for women.

- Department A had a 62% acceptance rate for men and an 82% acceptance rate for women!

  - 31% of men applied to Department A.
  - 6% of women applied to Department A.

- Department F had a 6% acceptance rate for men and a 7% acceptance rate for women!

  - 14% of men applied to Department F.
  - 19% of women applied to Department F.

- **Conclusion**: Women tended to apply to departments with a lower acceptance rate; the data don't support the hypothesis that there was major gender discrimination against women.


### Example: Restaurant reviews and phone types

- You are deciding whether to eat at Dirty Birds or The Loft.

- Suppose Yelp shows ratings aggregated by phone type (Android vs. iPhone).

| Phone Type | Stars for Dirty Birds | Stars for The Loft |
| ---------- | --------------------- | ------------------ |
| Android    | 4.24                  | 4.0                |
| iPhone     | 2.99                  | 2.79               |
| **All**    | **3.32**              | **3.37**           |

- **Question**: Should you choose Dirty Birds or The Loft?

- **Answer**: The type of phone you use likely has nothing to do with your taste in food – pick the restaurant that is rated higher overall.


### Rule of thumb 👍

- Let $(X, Y)$ be a pair of variables of interest. Simpson's paradox occurs when the association between $X$ and $Y$ reverses when we condition on $Z$, a third variable.

- If $Z$ has a **causal** connection to both $X$ and $Y$, we should condition on $Z$ and use deaggregated data.

- If not, we shouldn't condition on $Z$ and use the aggregated data instead.

- Berkeley gender discrimination: $X$ is gender, $Y$ is acceptance rate. $Z$ is the department.

  - $Z$ has a plausible causal effect on both $X$ and $Y$, so we should condition on $Z$.

- Yelp ratings: $X$ is the restaurant, $Y$ is the average stars. $Z$ is the phone type.
  - $Z$ doesn't plausibly cause $X$ to change, so we should not condition on $Z$.


### Takeaways

Be skeptical of...

- Aggregate statistics.
- People misusing statistics to "prove" that discrimination doesn't exist.
- Drawing conclusions from individual publications ($p$-hacking, publication bias, narrow focus, etc.).
- Everything!

**We need to apply domain knowledge and human judgement calls to decide what to do when Simpson's paradox is present.**


### Really?

To handle Simpson's paradox with rigor, we need some ideas from causal inference which we don't have time to cover in this course. This video has a good example of how to approach Simpson's paradox using a minimal amount of causal inference, if you're curious.


In [ ]:
IFrame(
    "https://www.youtube-nocookie.com/embed/zeuW1Z2EtLs?si=l2Dl7P-5RCq3ODpo",
    width=800,
    height=450,
)

### Further reading

- [What is Simpson's Paradox?](https://statisticsbyjim.com/basics/simpsons-paradox/)
- [Understanding Simpson's Paradox](https://ftp.cs.ucla.edu/pub/stat_ser/r414.pdf)
  - Requires more statistics background, but gives a rigorous understanding of when to use aggregated vs. unaggregated data.
